## Prerequisites

In [61]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.feature_selection import SelectKBest, f_classif
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier, export_text
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from sklearn.impute import SimpleImputer
from imblearn.over_sampling import SMOTE
import joblib
import warnings
warnings.filterwarnings('ignore')

# DATA PREPARATION

In [62]:
# Mount Google Drive in Colab
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [63]:
# Upload vegemite.csv to Colab first
df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/vegemite.csv')

print(f"Original dataset shape: {df.shape}")
print(f"Original class distribution:\n{df.iloc[:, -1].value_counts()}")

Original dataset shape: (15237, 47)
Original class distribution:
Class
2    7548
1    5047
0    2642
Name: count, dtype: int64


In [64]:
constant_columns = []
for col in df.columns[:-1]:  # Exclude target column
    if df[col].nunique() <= 1:  # Only one unique value (constant)
        constant_columns.append(col)
        print(f"Constant column found: {col} (unique values: {df[col].nunique()})")

if constant_columns:
    df = df.drop(columns=constant_columns)
    print(f"Removed {len(constant_columns)} constant columns")
else:
    print("No constant value columns found - Good!")

print(f"Dataset shape after constant column removal: {df.shape}")

Constant column found: TFE Steam temperature SP (unique values: 1)
Constant column found: TFE Product out temperature (unique values: 1)
Removed 2 constant columns
Dataset shape after constant column removal: (15237, 45)


In [65]:
# Shuffle and split data
df_shuffled = df.sample(frac=1, random_state=42).reset_index(drop=True)
target_col = df_shuffled.columns[-1]
X = df_shuffled.drop(target_col, axis=1)
y = df_shuffled[target_col]

X_train_temp, X_test, y_train_temp, y_test = train_test_split(
    X, y, test_size=1000, stratify=y, random_state=42
)
print(f"Test samples: {len(X_test)}, Training samples: {len(X_train_temp)}")

Test samples: 1000, Training samples: 14237


Does the dataset have any constant value columns?

In [66]:
categorical_threshold = 10
converted_cols = []

for col in X_train_temp.columns:
    unique_count = X_train_temp[col].nunique()
    if unique_count <= categorical_threshold and unique_count > 1:
        # Check if values are integers
        if X_train_temp[col].dtype in ['int64', 'int32'] or all(X_train_temp[col].dropna().apply(lambda x: float(x).is_integer())):
            converted_cols.append(col)
            X_train_temp[col] = X_train_temp[col].astype('category')
            X_test[col] = X_test[col].astype('category')
            print(f"Converted {col} to categorical (unique values: {unique_count})")

print(f"Total converted to categorical: {len(converted_cols)} columns")

Converted FFTE Feed tank level SP to categorical (unique values: 3)
Converted FFTE Pump 1 to categorical (unique values: 5)
Converted FFTE Pump 1 - 2 to categorical (unique values: 4)
Converted FFTE Pump 2 to categorical (unique values: 5)
Converted TFE Motor speed to categorical (unique values: 3)
Total converted to categorical: 5 columns


Check for missing values and handle them

In [67]:
# More robust missing value handling
missing_counts = X_train_temp.isnull().sum()
columns_with_missing = missing_counts[missing_counts > 0]

if len(columns_with_missing) > 0:
    print("Columns with missing values:")
    for col, count in columns_with_missing.items():
        percentage = (count / len(X_train_temp)) * 100
        print(f"  {col}: {count} missing values ({percentage:.2f}%)")

    print("\nApplying robust imputation for missing values...")

    # Separate numeric and categorical columns for imputation
    numeric_cols = X_train_temp.select_dtypes(include=[np.number]).columns
    categorical_cols = X_train_temp.select_dtypes(include=['category']).columns

    # Impute numeric columns with MEDIAN (more robust than mean for outliers)
    if len(numeric_cols) > 0:
        numeric_imputer = SimpleImputer(strategy='median')  # Changed from 'mean'
        X_train_temp[numeric_cols] = numeric_imputer.fit_transform(X_train_temp[numeric_cols])
        X_test[numeric_cols] = numeric_imputer.transform(X_test[numeric_cols])
        print(f"Applied median imputation to {len(numeric_cols)} numeric columns")

    # Impute categorical columns with most frequent
    if len(categorical_cols) > 0:
        categorical_imputer = SimpleImputer(strategy='most_frequent')
        X_train_temp[categorical_cols] = categorical_imputer.fit_transform(X_train_temp[categorical_cols])
        X_test[categorical_cols] = categorical_imputer.transform(X_test[categorical_cols])
        print(f"Applied mode imputation to {len(categorical_cols)} categorical columns")

    print("Enhanced missing value imputation completed!")
else:
    print("No missing values found - Dataset is clean!")
    print("Note: This is excellent data quality, but our pipeline is ready for missing values if they occur.")

No missing values found - Dataset is clean!
Note: This is excellent data quality, but our pipeline is ready for missing values if they occur.


Class balance analysis and correction

In [68]:
class_dist = y_train_temp.value_counts().sort_index()
print("\nClass distribution:")
for class_val, count in class_dist.items():
    percentage = (count / len(y_train_temp)) * 100
    print(f"  Class {class_val}: {count} samples ({percentage:.1f}%)")

imbalance_ratio = class_dist.max() / class_dist.min()
print(f"Class imbalance ratio: {imbalance_ratio:.2f}")

if imbalance_ratio > 1.5:
    print("Class imbalance detected (ratio > 1.5). Applying SMOTE...")
    # Convert categorical columns to numeric for SMOTE
    X_train_for_smote = X_train_temp.copy()
    categorical_cols = X_train_temp.select_dtypes(include=['category']).columns

    for col in categorical_cols:
        X_train_for_smote[col] = X_train_for_smote[col].cat.codes

    smote = SMOTE(random_state=42)
    X_train_balanced, y_train_balanced = smote.fit_resample(X_train_for_smote, y_train_temp)

    # Convert back to DataFrame
    X_train_balanced = pd.DataFrame(X_train_balanced, columns=X_train_temp.columns)

    print("After SMOTE balancing:")
    balanced_dist = pd.Series(y_train_balanced).value_counts().sort_index()
    for class_val, count in balanced_dist.items():
        percentage = (count / len(y_train_balanced)) * 100
        print(f"  Class {class_val}: {count} samples ({percentage:.1f}%)")
else:
    X_train_balanced, y_train_balanced = X_train_temp, y_train_temp
    print("Classes are reasonably balanced - no SMOTE needed")


Class distribution:
  Class 0: 2468 samples (17.3%)
  Class 1: 4716 samples (33.1%)
  Class 2: 7053 samples (49.5%)
Class imbalance ratio: 2.86
Class imbalance detected (ratio > 1.5). Applying SMOTE...
After SMOTE balancing:
  Class 0: 7053 samples (33.3%)
  Class 1: 7053 samples (33.3%)
  Class 2: 7053 samples (33.3%)


Feature exploration and composite features

In [69]:
# Convert categorical columns back to numeric for mathematical operations
categorical_cols = X_train_balanced.select_dtypes(include=['category']).columns
if len(categorical_cols) > 0:
    for col in categorical_cols:
        X_train_balanced[col] = X_train_balanced[col].cat.codes

# ALSO convert categorical columns in X_test to numeric
categorical_cols_test = X_test.select_dtypes(include=['category']).columns
if len(categorical_cols_test) > 0:
    for col in categorical_cols_test:
        X_test[col] = X_test[col].cat.codes

numeric_cols = X_train_balanced.select_dtypes(include=[np.number]).columns
composite_features = []

print("Creating composite features based on domain knowledge:")

# Create ratio features between SP and PV columns
sp_cols = [col for col in numeric_cols if 'SP' in col]
pv_cols = [col for col in numeric_cols if 'PV' in col]

print(f"Found {len(sp_cols)} SP columns and {len(pv_cols)} PV columns")

# SP to PV ratios (limited to avoid too many features)
for sp_col in sp_cols[:3]:
    base_name = sp_col.replace(' SP', '')
    pv_col = base_name + ' PV'
    if pv_col in pv_cols:
        ratio_col = f"{base_name}_SP_to_PV_ratio"
        X_train_balanced[ratio_col] = X_train_balanced[sp_col] / (X_train_balanced[pv_col] + 1e-8)
        X_test[ratio_col] = X_test[sp_col] / (X_test[pv_col] + 1e-8)
        composite_features.append(ratio_col)
        print(f"Created: {ratio_col}")

# Temperature difference features
temp_cols = [col for col in numeric_cols if 'Temperature' in col or 'temp' in col.lower()]
if len(temp_cols) >= 2:
    for i in range(min(2, len(temp_cols)-1)):
        diff_col = f"{temp_cols[i]}_minus_{temp_cols[i+1]}_diff"
        X_train_balanced[diff_col] = X_train_balanced[temp_cols[i]] - X_train_balanced[temp_cols[i+1]]
        X_test[diff_col] = X_test[temp_cols[i]] - X_test[temp_cols[i+1]]
        composite_features.append(diff_col)
        print(f"Created: {diff_col}")

# Flow rate efficiency features (if flow columns exist)
flow_cols = [col for col in numeric_cols if 'flow' in col.lower()]
if len(flow_cols) >= 2:
    efficiency_col = f"{flow_cols[0]}_to_{flow_cols[1]}_efficiency"
    X_train_balanced[efficiency_col] = X_train_balanced[flow_cols[0]] / (X_train_balanced[flow_cols[1]] + 1e-8)
    X_test[efficiency_col] = X_test[flow_cols[0]] / (X_test[flow_cols[1]] + 1e-8)
    composite_features.append(efficiency_col)
    print(f"Created: {efficiency_col}")

print(f"Total composite features created: {len(composite_features)}")

Creating composite features based on domain knowledge:
Found 9 SP columns and 11 PV columns
Created: FFTE Feed tank level_SP_to_PV_ratio
Created: FFTE Production solids_SP_to_PV_ratio
Created: FFTE Steam pressure_SP_to_PV_ratio
Created: FFTE Out steam temp SP_minus_FFTE Heat temperature 1_diff
Created: FFTE Heat temperature 1_minus_FFTE Heat temperature 2_diff
Created: TFE Out flow SP_to_FFTE Feed flow SP_efficiency
Total composite features created: 6


Final feature count

In [70]:
print(f"Final training set shape: {X_train_balanced.shape}")
print(f"Final test set shape: {X_test.shape}")
print(f"Total features in final dataset: {len(X_train_balanced.columns)}")

# Show feature breakdown
numeric_final = X_train_balanced.select_dtypes(include=[np.number]).columns
categorical_final = X_train_balanced.select_dtypes(include=['category']).columns
print(f"  - Numeric features: {len(numeric_final)}")
print(f"  - Categorical features: {len(categorical_final)}")
print(f"  - Original features: {len(X_train_balanced.columns) - len(composite_features)}")
print(f"  - Composite features: {len(composite_features)}")

Final training set shape: (21159, 50)
Final test set shape: (1000, 50)
Total features in final dataset: 50
  - Numeric features: 50
  - Categorical features: 0
  - Original features: 44
  - Composite features: 6


## Feature Selection, Model Training and Evaluation

In [71]:
# Ensure all data is numeric for feature selection
X_train_for_selection = X_train_balanced.copy()
X_test_for_selection = X_test.copy()

# Convert categorical columns to numeric codes if any remain
categorical_cols_final = X_train_for_selection.select_dtypes(include=['category']).columns
if len(categorical_cols_final) > 0:
    for col in categorical_cols_final:
        X_train_for_selection[col] = X_train_for_selection[col].cat.codes
        X_test_for_selection[col] = X_test_for_selection[col].cat.codes

# Final check for any remaining missing values
remaining_missing = X_train_for_selection.isnull().sum().sum()
if remaining_missing > 0:
    final_imputer = SimpleImputer(strategy='mean')
    X_train_for_selection = pd.DataFrame(
        final_imputer.fit_transform(X_train_for_selection),
        columns=X_train_for_selection.columns
    )
    X_test_for_selection = pd.DataFrame(
        final_imputer.transform(X_test_for_selection),
        columns=X_test_for_selection.columns
    )
    print("Final imputation completed!")
else:
    print("No missing values found after preprocessing!")

No missing values found after preprocessing!


Feature selection justification

In [72]:
print(f"Current feature count: {len(X_train_for_selection.columns)}")
print("Benefits of feature selection:")
print("  - Reduces overfitting (curse of dimensionality)")
print("  - Improves computational efficiency")
print("  - Enhances model interpretability")
print("  - Removes noise from irrelevant features")
print("3. SelectKBest with f_classif will identify most discriminative features")

# Apply feature selection
k_features = min(20, len(X_train_for_selection.columns))
print(f"Selecting top {k_features} features using ANOVA F-test (f_classif)...")

selector = SelectKBest(f_classif, k=k_features)
X_train_selected = selector.fit_transform(X_train_for_selection, y_train_balanced)
X_test_selected = selector.transform(X_test_for_selection)

selected_features = X_train_for_selection.columns[selector.get_support()]
print(f"Selected {len(selected_features)} features using SelectKBest")

# Show selected features with their scores
feature_scores = selector.scores_[selector.get_support()]
feature_ranking = pd.DataFrame({
    'Feature': selected_features,
    'F_Score': feature_scores
}).sort_values('F_Score', ascending=False)

print("\nTop selected features:")
print(feature_ranking.head(10))

# Convert back to DataFrames with proper column names
X_train_final = pd.DataFrame(X_train_selected, columns=selected_features)
X_test_final = pd.DataFrame(X_test_selected, columns=selected_features)

# Split for validation
X_train_split, X_val_split, y_train_split, y_val_split = train_test_split(
    X_train_final, y_train_balanced, test_size=0.2, stratify=y_train_balanced, random_state=42
)

Current feature count: 50
Benefits of feature selection:
  - Reduces overfitting (curse of dimensionality)
  - Improves computational efficiency
  - Enhances model interpretability
  - Removes noise from irrelevant features
3. SelectKBest with f_classif will identify most discriminative features
Selecting top 20 features using ANOVA F-test (f_classif)...
Selected 20 features using SelectKBest

Top selected features:
                                              Feature     F_Score
2                                     TFE Out flow SP  540.398462
12                             FFTE Temperature 3 - 2  523.700497
9                              FFTE Temperature 1 - 1  523.394607
11                             FFTE Temperature 2 - 1  445.608422
15                                    TFE Temperature  401.884428
10                             FFTE Temperature 1 - 2  381.049580
18  FFTE Heat temperature 1_minus_FFTE Heat temper...  305.126607
1                           FFTE Production solids S

Train multiple ML models

In [73]:
print("\n7) Training 6 different ML models with optimized hyperparameters:")

models = {
    'DecisionTree': DecisionTreeClassifier(random_state=42, max_depth=10, min_samples_split=20, min_samples_leaf=10),
    'RandomForest': RandomForestClassifier(random_state=42, n_estimators=100, max_depth=15),
    'SVM': SVC(random_state=42, C=1.0),
    'LogisticRegression': LogisticRegression(random_state=42, max_iter=1000),
    'KNN': KNeighborsClassifier(n_neighbors=5),
    'NaiveBayes': GaussianNB()
}


7) Training 6 different ML models with optimized hyperparameters:


Model evaluation and comparison

In [74]:
results = {}
trained_models = {}

for name, model in models.items():
    model.fit(X_train_split, y_train_split)
    trained_models[name] = model

    y_pred_val = model.predict(X_val_split)
    val_accuracy = accuracy_score(y_val_split, y_pred_val)
    cv_scores = cross_val_score(model, X_train_final, y_train_balanced, cv=5, scoring='accuracy')

    results[name] = {
        'val_accuracy': val_accuracy,
        'cv_mean': cv_scores.mean(),
        'cv_std': cv_scores.std(),
        'classification_report': classification_report(y_val_split, y_pred_val),
        'confusion_matrix': confusion_matrix(y_val_split, y_pred_val)
    }

In [75]:
# Create comparison table
comparison_df = pd.DataFrame({
    'Model': list(models.keys()),
    'Validation_Accuracy': [results[name]['val_accuracy'] for name in models.keys()],
    'CV_Mean': [results[name]['cv_mean'] for name in models.keys()],
    'CV_Std': [results[name]['cv_std'] for name in models.keys()]
}).set_index('Model')

print("8-9) Model Performance Comparison:")
print(comparison_df.sort_values('Validation_Accuracy', ascending=False))

8-9) Model Performance Comparison:
                    Validation_Accuracy   CV_Mean    CV_Std
Model                                                      
RandomForest                   0.994565  0.989555  0.005242
KNN                            0.935491  0.926556  0.003087
DecisionTree                   0.907609  0.889409  0.015974
LogisticRegression             0.485350  0.483624  0.006741
NaiveBayes                     0.479442  0.467035  0.003730
SVM                            0.456522  0.437969  0.006833


Best model selection and justification

In [76]:
best_model_name = comparison_df['Validation_Accuracy'].idxmax()
best_model = trained_models[best_model_name]
print(f"\n10) Best model selection:")
print(f"Selected model: {best_model_name}")
print(f"Validation accuracy: {comparison_df.loc[best_model_name, 'Validation_Accuracy']:.4f}")

print(f"\nJustification for selecting {best_model_name}:")
print("- Highest validation accuracy")
print("- Consistent cross-validation performance")
print("- Good generalization capability")

print(f"\nClassification Report for {best_model_name}:")
print(results[best_model_name]['classification_report'])


10) Best model selection:
Selected model: RandomForest
Validation accuracy: 0.9946

Justification for selecting RandomForest:
- Highest validation accuracy
- Consistent cross-validation performance
- Good generalization capability

Classification Report for RandomForest:
              precision    recall  f1-score   support

           0       1.00      0.99      1.00      1410
           1       0.99      0.99      0.99      1411
           2       1.00      1.00      1.00      1411

    accuracy                           0.99      4232
   macro avg       0.99      0.99      0.99      4232
weighted avg       0.99      0.99      0.99      4232



Save model

In [77]:
# Save Model
joblib.dump(best_model, 'best_vegemite_model.pkl')
joblib.dump(selector, 'feature_selector.pkl')
print("11) Model saved successfully")

11) Model saved successfully


## ML TO AI DEPLOYMENT

Load model and test on unseen data

In [78]:
# Load Model and Test on Unseen Data
loaded_model = joblib.load('best_vegemite_model.pkl')
loaded_selector = joblib.load('feature_selector.pkl')

# Process test data through the same pipeline
X_test_processed = X_test_for_selection.copy()

# Apply feature selection
X_test_selected = loaded_selector.transform(X_test_processed)
X_test_final_processed = pd.DataFrame(X_test_selected, columns=selected_features)

Performance measurement on unseen data

In [81]:
y_pred_test = loaded_model.predict(X_test_final_processed)
test_accuracy = accuracy_score(y_test, y_pred_test)

print(f"12-16) Performance on 1000 unseen data points:")
print(f"Test accuracy: {test_accuracy:.4f}")
print("\nClassification Report on Test Data:")
print(classification_report(y_test, y_pred_test))

print("\nConfusion Matrix on Test Data:")
print(confusion_matrix(y_test, y_pred_test))

12-16) Performance on 1000 unseen data points:
Test accuracy: 0.9870

Classification Report on Test Data:
              precision    recall  f1-score   support

           0       0.98      0.99      0.99       174
           1       0.97      0.99      0.98       331
           2       1.00      0.98      0.99       495

    accuracy                           0.99      1000
   macro avg       0.98      0.99      0.99      1000
weighted avg       0.99      0.99      0.99      1000


Confusion Matrix on Test Data:
[[173   1   0]
 [  2 329   0]
 [  2   8 485]]


Compare all models on test data

In [82]:
print(f"\n17) All models test performance:")
test_results = {}
for name, model in trained_models.items():
    y_pred = model.predict(X_test_final_processed)
    accuracy = accuracy_score(y_test, y_pred)
    test_results[name] = accuracy
    print(f"{name}: {accuracy:.4f}")

print("\nModel Ranking Consistency Analysis:")
print("Comparing validation vs test performance to ensure robust model selection\n")

# Get validation rankings
val_ranking = comparison_df.sort_values('Validation_Accuracy', ascending=False)
print("VALIDATION RANKING:")
for i, (model, row) in enumerate(val_ranking.iterrows(), 1):
    print(f"{i}. {model}: {row['Validation_Accuracy']:.4f}")

print(f"\nTEST RANKING:")
test_ranking = sorted([(name, acc) for name, acc in test_results.items()],
                     key=lambda x: x[1], reverse=True)
for i, (model, acc) in enumerate(test_ranking, 1):
    print(f"{i}. {model}: {acc:.4f}")

# Check if top model is consistent
val_top = val_ranking.index[0]
test_top = test_ranking[0][0]

print(f"\nConsistency Check:")
print(f"Best on Validation: {val_top}")
print(f"Best on Test: {test_top}")

if val_top == test_top:
    print("EXCELLENT: Same model ranks #1 on both validation and test!")
    print("This indicates robust model selection and good generalization.")
else:
    print("CAUTION: Different models rank #1 on validation vs test")
    print("This could indicate overfitting or high variance in model performance")

# Calculate rank correlation for robustness
val_scores = [comparison_df.loc[name, 'Validation_Accuracy'] for name in comparison_df.index]
test_scores = [test_results[name] for name in comparison_df.index]
correlation = np.corrcoef(val_scores, test_scores)[0,1]
print(f"\nValidation-Test Correlation: {correlation:.3f}")
if correlation > 0.8:
    print("Strong correlation - reliable model ranking")
elif correlation > 0.6:
    print("Moderate correlation - acceptable ranking reliability")
else:
    print("Weak correlation - consider more robust validation")


17) All models test performance:
DecisionTree: 0.9070
RandomForest: 0.9870
SVM: 0.3820
LogisticRegression: 0.4660
KNN: 0.9190
NaiveBayes: 0.4310

Model Ranking Consistency Analysis:
Comparing validation vs test performance to ensure robust model selection

VALIDATION RANKING:
1. RandomForest: 0.9946
2. KNN: 0.9355
3. DecisionTree: 0.9076
4. LogisticRegression: 0.4853
5. NaiveBayes: 0.4794
6. SVM: 0.4565

TEST RANKING:
1. RandomForest: 0.9870
2. KNN: 0.9190
3. DecisionTree: 0.9070
4. LogisticRegression: 0.4660
5. NaiveBayes: 0.4310
6. SVM: 0.3820

Consistency Check:
Best on Validation: RandomForest
Best on Test: RandomForest
EXCELLENT: Same model ranks #1 on both validation and test!
This indicates robust model selection and good generalization.

Validation-Test Correlation: 0.998
Strong correlation - reliable model ranking


DEVELOP RULES FROM ML MODEL

In [83]:
print("\nCLARIFICATION: SP vs PV Features")
print("SP = Set Points (controllable parameters that humans can adjust)")
print("PV = Process Variables (machine-generated measurements, not directly controllable)")
print("Goal: Extract control rules using only SP features for operational guidance\n")

# Extract SP features from selected features first
sp_features_selected = [col for col in selected_features if 'SP' in col]
print(f"SP features found in TOP selected features: {len(sp_features_selected)}")

if sp_features_selected:
    print("Using SP features from feature selection (most predictive SP features):")
    for feature in sp_features_selected:
        print(f"  {feature}")

    # Use selected SP features for rules
    X_train_sp = X_train_final[sp_features_selected]
    sp_features_for_rules = sp_features_selected

else:
    print("No SP features in top selected features.")
    print("Extracting ALL available SP features from dataset for rule generation...")

    # Get all SP features from the balanced training set
    all_sp_features = [col for col in X_train_balanced.columns if 'SP' in col]
    print(f"Total SP features available in dataset: {len(all_sp_features)}")

    if all_sp_features:
        print("Available SP features for control rules:")
        for feature in all_sp_features:
            print(f"  {feature}")

        # Use all SP features since none were selected as most predictive
        X_train_sp = X_train_balanced[all_sp_features]
        sp_features_for_rules = all_sp_features
    else:
        print("No SP features found in dataset!")
        sp_features_for_rules = []

# Generate decision rules if SP features exist
if len(sp_features_for_rules) > 0:
    print(f"\nGENERATING CONTROL RULES using {len(sp_features_for_rules)} SP features:")

    # Train decision tree on SP features only for rule extraction
    dt_sp = DecisionTreeClassifier(
        random_state=42,
        max_depth=5,           # Limit depth for interpretable rules
        min_samples_split=50,  # Avoid overfitting
        min_samples_leaf=25    # Ensure statistical significance
    )
    dt_sp.fit(X_train_sp, y_train_balanced)

    # Export interpretable tree rules
    tree_rules = export_text(dt_sp, feature_names=sp_features_for_rules, max_depth=5)
    print("\nDECISION TREE CONTROL RULES:")
    print("="*80)
    print(tree_rules)
    print("="*80)

    # Feature importance for control priorities
    importance_df = pd.DataFrame({
        'SP_Feature': sp_features_for_rules,
        'Control_Importance': dt_sp.feature_importances_
    }).sort_values('Control_Importance', ascending=False)

    print("\nSP FEATURE IMPORTANCE FOR PROCESS CONTROL:")
    print(importance_df)

    # Generate practical control recommendations
    print(f"\nPRACTICAL CONTROL RECOMMENDATIONS:")
    print("Based on decision tree analysis, here are the recommended SP ranges:\n")

    for class_label in sorted(y_train_balanced.unique()):
        print(f"FOR CLASS {class_label} (Target Consistency Level):")
        class_mask = y_train_balanced == class_label

        if class_mask.sum() > 0:
            # Get samples for this class
            class_samples = X_train_sp.loc[class_mask]

            # Top 3 most important SP features for control
            top_sp_features = importance_df.head(3)['SP_Feature'].tolist()

            for feature in top_sp_features:
                if feature in class_samples.columns:
                    values = class_samples[feature]
                    if len(values) > 0:
                        q25, q50, q75 = values.quantile([0.25, 0.5, 0.75])
                        print(f"  {feature}")
                        print(f"    Recommended range: {q25:.2f} to {q75:.2f}")
                        print(f"    Optimal target: {q50:.2f} (median)")
        print()

    print("Control rules successfully generated!")

else:
    print("Cannot generate SP-based control rules - no SP features available")
    print("This suggests the process may be primarily driven by measured variables (PV) rather than set points")


CLARIFICATION: SP vs PV Features
SP = Set Points (controllable parameters that humans can adjust)
PV = Process Variables (machine-generated measurements, not directly controllable)
Goal: Extract control rules using only SP features for operational guidance

SP features found in TOP selected features: 7
Using SP features from feature selection (most predictive SP features):
  FFTE Feed tank level SP
  FFTE Production solids SP
  TFE Out flow SP
  TFE Vacuum pressure SP
  FFTE Feed flow SP
  FFTE Out steam temp SP_minus_FFTE Heat temperature 1_diff
  TFE Out flow SP_to_FFTE Feed flow SP_efficiency

GENERATING CONTROL RULES using 7 SP features:

DECISION TREE CONTROL RULES:
|--- FFTE Feed flow SP <= 10197.65
|   |--- FFTE Feed flow SP <= 9233.66
|   |   |--- TFE Out flow SP <= 2170.56
|   |   |   |--- FFTE Feed tank level SP <= 1.50
|   |   |   |   |--- TFE Out flow SP_to_FFTE Feed flow SP_efficiency <= 0.21
|   |   |   |   |   |--- class: 1
|   |   |   |   |--- TFE Out flow SP_to_FFTE F